# Basin stats examples

In [11]:
import datetime
import geopandas as gpd
import pandas as pd

import rabpro
from rabpro.basin_stats import Dataset

In [6]:
# # Authenticate earthengine if necessary
# import ee
# ee.Authenticate()
# ee.Initialize()

In [9]:
%%capture
coords = (56.22659, -130.87974)
rpo = rabpro.profiler(coords, name='basic_test')
rpo.delineate_basin()
gdf = rpo.watershed

### Categorical asset - a single time window

In [12]:
urls, tasks = rabpro.basin_stats.compute(
        [Dataset("MODIS/006/MCD12Q1", "LC_Type1", stats=["freqhist"], start="2010-01-01", end="2011-01-01")],
        basins_gdf=gdf        
    )
data = rabpro.basin_stats.format_gee(urls, ["lulc"])
res = gpd.GeoDataFrame(pd.concat([data, gdf], axis=1))
res.head()

,lulc_da_km2,lulc_1,lulc_10,lulc_11,lulc_15,lulc_16,lulc_17,lulc_4,lulc_8,lulc_9,da_km2,geometry
0,2131.577825,2145.462745,2824.396078,212.0,2965.690196,1208.223529,9.0,58.0,454.964706,3.0,2131.577825,"POLYGON ((-130.98238 56.13723, -130.98395 56.1..."


### Numeric asset - multiple time windows

In [13]:
urls, tasks = rabpro.basin_stats.compute(
        [Dataset("MODIS/006/MOD17A3HGF", "Npp")],
        basins_gdf=gdf
    )
data = rabpro.basin_stats.format_gee(urls, ["npp"])
data["year"] = [x for x in range(2000, int(datetime.date.today().strftime("%Y")))]
res = gpd.GeoDataFrame(pd.concat([data, gdf], axis=1))
res.geometry[res.geometry.isna()] = res.geometry[0]
res.head()

,npp_da_km2,npp_mean,year,da_km2,geometry
0,2131.577825,1747.866809,2000,2131.577825,"POLYGON ((-130.98238 56.13723, -130.98395 56.1..."
1,2131.577825,1597.141272,2001,NaN,"POLYGON ((-130.98238 56.13723, -130.98395 56.1..."
2,2131.577825,1808.514660,2002,NaN,"POLYGON ((-130.98238 56.13723, -130.98395 56.1..."
3,2131.577825,2401.387620,2003,NaN,"POLYGON ((-130.98238 56.13723, -130.98395 56.1..."
4,2131.577825,2855.209354,2004,NaN,"POLYGON ((-130.98238 56.13723, -130.98395 56.1..."


## Numeric asset - time averaging

In [14]:
urls, tasks = rabpro.basin_stats.compute(
        [Dataset("MODIS/006/MOD17A3HGF", "Npp", time_stats=["median"])],
        basins_gdf=gdf
    )
data = rabpro.basin_stats.format_gee(urls, ["npp"])
res = gpd.GeoDataFrame(pd.concat([data, gdf], axis=1))
res.head()

,npp_da_km2,npp_mean,da_km2,geometry
0,2131.577825,2289.570271,2131.577825,"POLYGON ((-130.98238 56.13723, -130.98395 56.1..."
